In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

from scipy.stats import uniform, randint, loguniform
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor

from DataTransformer import DataTransformer
from TrainUtils import *


%load_ext autoreload
%autoreload 2

In [2]:
train_df, target = load_data()

transformer = DataTransformer()
X = transformer.fit_transform(train_df, obj_to_num=True)
X = X.iloc[np.random.choice(np.arange(len(X)), 300, False)]
y = target[X.index]


print(f"Is there nan: {np.any(X.isnull())}")

Is there nan: False


In [3]:
parameters = {
    "learning_rate": loguniform(5e-3,  1e-1),
    "max_depth": randint(1, 500),
    "n_estimators": randint(5000, 15000),
    "lambda" : loguniform(1e-4, 10),
    "subsample" : uniform(0.4, 1),
    "min_child_weight" : loguniform(0.1, 55),
    "gamma" : loguniform(1e-4, 10)
    }
grid_model = RandomizedSearchCV(XGBRegressor(seed=0), parameters, cv=3, n_jobs=-1)
grid_model.fit(X, y);

C:\Users\georg\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.82574616 0.80043656        nan 0.81607069
        nan 0.80833979        nan 0.81734095]
  warnings.warn(


In [4]:
print(grid_model.best_params_)

gs_model = XGBRegressor(**grid_model.best_params_, seed=0)

train_df, target = load_data()
X = transformer.fit_transform(train_df, obj_to_num=True)
y = target
gs_model.fit(X, y);

{'gamma': 0.0042837547518491965, 'lambda': 0.0029124722325491285, 'learning_rate': 0.024167616986944442, 'max_depth': 444, 'min_child_weight': 1.9615345776352848, 'n_estimators': 9919, 'subsample': 0.6701046206512675}


In [5]:
submission(transformer=transformer, gs_model=gs_model, obj_to_num=True)

RMSLE submission: 0.13101237547992442


In [6]:
0.12291506349262009

0.12291506349262009